ETL <br>
En esta notebook tendrán todo el proceso de extracción, transformación y carga(ETL) del dataframe user_reviews.json.

In [1]:
import pandas as pd
import numpy as np
import json 
import ast

Extracción de la data proveniente del archivo json

Se crea una variable para almacenar la ruta del archivo user_reviews.json para que su posterior uso ayude a tener  un codigo más legible.

In [2]:
reviews = "C:\\Users\\saulz\\proyecto_individual\\Dataset\\user_reviews.json\\australian_user_reviews.json"

-Comienza la extracción de los datos para cada dataframe respectivamente. <br>
-Se crea una lista vacia para almacenar la información del archivo. <br>
-La lista se transforma en dataframes usando la libreria Pandas. <br>

In [3]:
lista = []

with open (reviews, encoding='utf-8') as file_reviews:
    for line in file_reviews.readlines():
        lista.append(ast.literal_eval(line))

reviewsdf = pd.DataFrame(lista)

Transformación de los datos. <br>
-Se realizan las siguientes transformaciones con la finalidad de tener un df menos pesado y unicamente con los datos necesarios. <br>
-Se crea una mascara para hacer más facil la corroboración de datos. <br>
-Se verifican si existen datos anidados en el df. <br>
-Se eliman las siguientes columnas: 'user_url', 'funny', 'posted', 'last_edited', 'reviews', 'helpful'. <br>

In [4]:
mascara = reviewsdf['user_id'] == reviewsdf ['user_url']
filas_mascara = reviewsdf[mascara]
filas_mascara.head(5)

,user_id,user_url,reviews


Verificación de la existencia de datos anidados en el dataframe

In [5]:
if any (isinstance(valor,(list,dict,tuple))for valor in reviewsdf['reviews']):
    print('La columna tiene datos anidados')
else: 
    print('La columna no tiene datos anidados')

La columna tiene datos anidados


-Se desanidan los datos <br>
-Se crean variables para desanidar los datos desanidados. <br>
-Se reorganiza los datos desanidados en la variables. <br>
-Se unen ambas varibales para la creación de un df reorganizado. <br>

In [6]:

reviewsdf_expandido = reviewsdf.explode('reviews')
reviewsdf_desanidado = pd.json_normalize(reviewsdf_expandido['reviews'])

reviewsdf_expandido = reviewsdf_expandido.reset_index(drop=True)
reviewsdf_desanidado = reviewsdf_desanidado.reset_index(drop=True)

reviewsdf_final = pd.concat([reviewsdf_expandido, reviewsdf_desanidado], axis=1)
reviewsdf_final.head(5)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Se realiza un cheque de la información del dataframe una vez que ya se desanido 

In [7]:
reviewsdf_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   reviews      59305 non-null  object
 3   funny        59305 non-null  object
 4   posted       59305 non-null  object
 5   last_edited  59305 non-null  object
 6   item_id      59305 non-null  object
 7   helpful      59305 non-null  object
 8   recommend    59305 non-null  object
 9   review       59305 non-null  object
dtypes: object(10)
memory usage: 4.5+ MB


Se procede a la eliminación de las columnas que no serán relevantes para el proceso.

In [8]:
reviewsdf_final.drop(['user_url', 'funny', 'posted', 'last_edited', 'reviews', 'helpful'], axis=1, inplace=True)
reviewsdf_final

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,True,It's unique and worth a playthrough.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,True,I know what you think when you see this title ...
4,js41637,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,730,True,:D


Se almacena el dataframe en un archivo csv para usarlo en la etapa de EDA

In [9]:
ruta= "C:/Users/saulz/proyecto_individual/Dataset csv"
reviewsdf_final.to_csv('reviewsdf_final.csv', index=False)